### Per poter scaricare il dataset dobbiamo partire con questo comando
## !pip install kaggle --upgrade

In [ ]:
# Per via di un bug di Colab, avviare la seconda cella (darà errore) e poi partire da qui e runnare la seconda di nuovo dopo questa e dovrebbe funzionare tutto
!cp kaggle.json ~/.kaggle/

In [ ]:
# Facciamo il login tramite token su Kaggle dopo aver scaricato le API ed aver messo il file "kaggle.json" nel giusto percorso ( su windows "C:\\Users\\{nomeutente}\\.kaggle\\")

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [ ]:
# scarichiamo il dataset

!kaggle competitions download -c dogs-vs-cats

In [ ]:
# Import

import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers



In [ ]:
# Spacchettiamo  il dataset dallo zip

main = "dogs-vs-cats.zip"
train = "train.zip"
test = "test1.zip"
cartella_principale = "DOGvsCATS/"
files = "dataset/"

import zipfile

if os.path.isfile(train) and os.path.isfile(test) == True:
    pass
else:
    with zipfile.ZipFile(main,"r") as zipp:
        zipp.extractall()
    if os.path.isdir("train") and os.path.isdir("test1") == True:
        pass
    else:
        with zipfile.ZipFile(train, "r") as zipp:
            zipp.extractall(files)

        with zipfile.ZipFile(test, "r") as zipp:
            zipp.extractall(files)

In [ ]:
# Creiamo un dataframe per poter modellare i dati

dataset_train_path = "dataset/train/"

nomefile = os.listdir(dataset_train_path)
specie = [x.split(".")[0] for x in nomefile]

dati = pd.DataFrame({"Nome_File": nomefile, "Specie":specie})

#dati["Specie"] = dati["Specie"].replace(["cat","dog"],[0,1])


dati

In [ ]:

train_path = "working/train/"
test_path = "working/test/"

# Separiamo le immagini per cani e gatti
from os import makedirs, listdir
from random import random
from shutil import copyfile

cartel= [train_path, test_path]
# Creazione sottocartelle
for i in cartel:
    nome_cartel=["/cani/","/gatti/"]
    for j in nome_cartel:
        nuova_cartel = i+j
        makedirs(nuova_cartel, exist_ok=True)

# Percentuale di immagini per validare il training
valutazione = 0.2

# Copia delle immagini nelle sottocartelle
src_dir = "dataset/"
source = "dataset/train/"
for immagine in listdir(source):
    src=src_dir+"train/"+immagine
    destinazione="working/train/"
    if random()< valutazione:
        destinazione="working/test/"
    if immagine.startswith("cat"):
        dest = destinazione + "gatti/" + immagine
        copyfile(src, dest)
    elif immagine.startswith("dog"):
        dest = destinazione + "cani/" + immagine
        copyfile(src, dest)


path1 = "working/train/gatti"
path2= "working/train/cani"
path3= "working/test/gatti"
path4= "working/test/cani"

print('Ci sono ' ,len(os.listdir(path1)),' immagini di gatti in training')
print('Ci sono ' ,len(os.listdir(path2)),' immagini di cani in training')
print('Ci sono ' ,len(os.listdir(path3)),' immagini di gatti in test')
print('Ci sono ' ,len(os.listdir(path4)),' immagini di cani in test')

In [ ]:
# Settiamo i parametri
grandezza_immagine = 256
grandezza_batch = 32

# Creiamo i generatori

train_gen = keras.utils.image_dataset_from_directory(
    directory= "working/train",
    labels="inferred",
    label_mode="binary",
    batch_size=grandezza_batch,
    image_size=(grandezza_immagine,grandezza_immagine)
)

test_gen = keras.utils.image_dataset_from_directory(
    directory= "working/test",
    labels="inferred",
    label_mode="binary",
    batch_size=grandezza_batch,
    image_size=(grandezza_immagine,grandezza_immagine)
)

In [ ]:
# Creiamo il modello sequenziale
modello = keras.Sequential()
modello.add(layers.Conv2D(32, (3,3), input_shape=(grandezza_immagine,grandezza_immagine,3), activation="relu"))
modello.add(layers.BatchNormalization())
modello.add(layers.MaxPooling2D((2,2)))
modello.add(layers.Conv2D(64,(3,3),activation="relu"))
modello.add(layers.BatchNormalization())
modello.add(layers.MaxPooling2D((2,2)))
modello.add(layers.Conv2D(128,(3,3),activation="relu"))
modello.add(layers.BatchNormalization())
modello.add(layers.MaxPooling2D((2,2)))

modello.add(layers.Flatten())
modello.add(layers.Dense(512, activation="relu"))
modello.add(layers.Dropout(0.2))
modello.add(layers.Dense(1, activation="sigmoid"))



In [ ]:
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',
                                            patience=2,
                                            factor=0.5,
                                            min_lr = 0.00001,
                                            verbose = 1)

earlystop = EarlyStopping(monitor = 'val_loss',
                          min_delta = 0,
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

In [ ]:
# Compiliamo il modello
modello.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])

modello.summary()

In [ ]:
# Addestriamo il modello
history = modello.fit(train_gen,
                      callbacks=[earlystop,learning_rate_reduction],
                      epochs = 10,
                      validation_data=test_gen)

In [ ]:
plt.plot(history.history["accuracy"], color="red", label="TRAIN")
plt.plot(history.history["val_accuracy"], color="blue", label="TEST")
plt.title("Accuracy")
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history["loss"], color="red", label="TRAIN")
plt.plot(history.history["val_loss"], color="blue", label="TEST")
plt.title("Loss")
plt.legend()
plt.show()

In [ ]:
test_immagine = cv2.imread("dataset/test1/3.jpg")
plt.imshow(test_immagine)

In [ ]:
test_immagine.shape

In [ ]:
test_immagine = cv2.resize(test_immagine,(256,256))
input = test_immagine.reshape((1,256,256,3))

In [ ]:
predizione = modello.predict(input)
if predizione > 0.5 :
  print("Gatto")
elif predizione <0.5:
  print("Cane")